# Python Mixins

* A `mixin` is a `class` that provides method implementations for reuse by multiple related child classes.
* A `mixin` doesn’t define a new type. Therefore, it is not intended for direct instantiation.
* It’s a convention in Python to name `mixin` classes with the suffix **`Mixin`**.

In [1]:
import json

import numpy as np
import pandas as pd
from pydantic import BaseModel, validator, ValidationError
from typing import Optional, Sequence, Union
from pprint import pprint

# Custom import
from src.data_manager import load_data

# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600
%load_ext lab_black

%load_ext autoreload
%autoreload 2

In [2]:
class Person:
    """This is the base class."""

    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age


# A class that inherits form the Person class
class Employee(Person):
    def __init__(self, name: str, age: int, skills: Sequence, dependents: dict) -> None:
        super().__init__(name, age)
        self.skills = skills
        self.dependents = dependents

    def __repr__(self) -> str:
        return (
            f"{self.__class__.__name__}(name={self.name}, "
            f"age={self.age}, "
            f"skills={self.skills}, "
            f"dependents={self.dependents})"
        )

#### Create an instance

In [3]:
emp_1 = Employee(
    name="John",
    age=30,
    skills=["Data Science", "Statistics", "Maths", "Software Engineering"],
    dependents={"wife": "Jane", "children": ["Alice", "Bob"]},
)
print(emp_1)

Employee(name=John, age=30, skills=['Data Science', 'Statistics', 'Maths', 'Software Engineering'], dependents={'wife': 'Jane', 'children': ['Alice', 'Bob']})


#### Create Mixins

* If we want to convert the `Employee` object to a dictionary, we can add a new method to the `Employee` class, which converts the object to a dictionary.

* However, we may want to convert objects of other classes to dictionaries. To make the code reusable, we can define a `mixin` class called **`DictMixin`** shown below.
* A `mixin` for creating JSON objects, `JSONMixin` will also be added to the `Employee` class.

**Note:**
The `__dict__` in Python represents a `dictionary` or any `mapping` object that is used to store the `attributes` of the object. 

In [4]:
class DictMixin:
    """This is a mixin for creating Python dictionaries."""

    def to_dict(self) -> dict:
        """This converts the object to a Python dict."""
        return self._traverse_dict(self.__dict__)

    def _traverse_dict(self, attributes: dict) -> dict:
        """This helper function is used to create a Python dict."""
        result = {}

        for key, value in attributes.items():
            result[key] = self._traverse(value)

        return result

    def _traverse(self, value: Union[Sequence, int, str, None]):
        """This helper function returns a Python dict."""
        # if it's an instance of DictMixin
        if isinstance(value, self.__class__):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        # if it's a class
        elif hasattr(value, "__dict__"):
            return self._traverse_dict(value.__dict__)
        else:
            return value


class JSONMixin:
    """This is a mixin for converting an object into a JSON (string)."""

    def to_json(self) -> str:
        """This returns a JSON string."""
        return json.dumps(self.__dict__)

In [5]:
class Employee(Person, DictMixin, JSONMixin):
    def __init__(
        self, name: str, age: int, skills: Sequence, dependents: dict, **kwargs
    ) -> None:
        super().__init__(name, age)
        self.skills = skills
        self.dependents = dependents
        self.kwargs = kwargs

    def __repr__(self) -> str:
        return (
            f"{self.__class__.__name__}(name={self.name}, "
            f"age={self.age}, "
            f"skills={self.skills}, "
            f"dependents={self.dependents}, "
            f"kwargs={self.kwargs})"
        )

In [6]:
emp_2 = Employee(
    name="Philip",
    age=30,
    skills=["Software Engineering", "Communication"],
    dependents={"brother": "Mark"},
    Location={"Continent": "Nigeria"},
)
emp_2

Employee(name=Philip, age=30, skills=['Software Engineering', 'Communication'], dependents={'brother': 'Mark'}, kwargs={'Location': {'Continent': 'Nigeria'}})

In [7]:
emp_2.to_dict()

{'name': 'Philip',
 'age': 30,
 'skills': ['Software Engineering', 'Communication'],
 'dependents': {'brother': 'Mark'},
 'kwargs': {'Location': {'Continent': 'Nigeria'}}}

In [8]:
pprint(emp_2.to_json(), indent=2)

('{"name": "Philip", "age": 30, "skills": ["Software Engineering", '
 '"Communication"], "dependents": {"brother": "Mark"}, "kwargs": {"Location": '
 '{"Continent": "Nigeria"}}}')
